<a href="https://colab.research.google.com/github/itsjafer/decktor-cli/blob/main/notebooks/quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeckTor Quickstart

Improve your Anki decks using Google's Gemini API directly in your browser. No coding required!


> **⚠️ IMPORTANT:** Your deck must contain **only one type of card** (either all Basic or all Cloze). Select the matching card type below.

In [ ]:
#@title 1. Install & Setup
#@markdown Run this cell to install DeckTor and configure your API key.

import os
from IPython.display import clear_output

# Install package
!pip install git+https://github.com/itsjafer/decktor-cli.git
clear_output()
print("✅ Installation complete!")

# API Key Setup
print("\n--- Optional: Gemini API Key ---")
print("If you want to use official Gemini models (faster/more stable), enter your key below.")
print("If you want to use the GPT4Free (Free), just press Enter or leave the field empty.")
GOOGLE_API_KEY = "Enter your key here (or leave blank for Free model)" #@param {type:"string"}

if GOOGLE_API_KEY and GOOGLE_API_KEY != "Enter your key here (or leave blank for Free model)":
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ API Key configured!")
else:
    print("⚠️  No API Key provided. You can only use 'GPT4Free (Free)'.")

In [ ]:
#@title 2. Upload Your Deck
#@markdown Click the play button to upload your `.apkg` file.

from google.colab import files

uploaded = files.upload()
input_filename = next(iter(uploaded))
print(f"\n✅ Uploaded: {input_filename}")

In [ ]:
#@title 3. Run DeckTor
#@markdown Choose your model, card type, and start processing.
#@markdown 
#@markdown **Card Type:** Select the type that matches ALL cards in your deck.
#@markdown - **Basic (Front/Back):** Standard question/answer cards
#@markdown - **Cloze:** Cards with {{c1::cloze deletions}}

output_filename = "improved_" + input_filename

# Configuration Options
model = "Gemini 2.5 Flash Lite" #@param ["Gemini 2.5 Flash Lite", "Gemini 2.5 Flash", "GPT-5 Mini", "GPT-5", "Claude Haiku 4.5", "Claude Sonnet 4.5", "GPT4Free (Free)"]
card_type = "Basic (Front/Back)" #@param ["Basic (Front/Back)", "Cloze"]
batch_size = 10 #@param {type:"integer"}
limit = 10 #@param {type:"integer"}

# Auto-switch to Free model if no API Key is set
import os
if not os.environ.get("GOOGLE_API_KEY"):
    if model != "GPT4Free (Free)":
        print("ℹ️ No API Key found. Switching to GPT4Free (Free) model.")
        model = "GPT4Free (Free)"

# Get the prompt file path based on card type
import subprocess
result = subprocess.run(['python', '-c', 'import decktor; import os; print(os.path.dirname(decktor.__file__))'], capture_output=True, text=True)
decktor_path = result.stdout.strip()

if card_type == "Basic (Front/Back)":
    prompt_file = os.path.join(decktor_path, 'prompts', 'default.txt')
else:
    prompt_file = os.path.join(decktor_path, 'prompts', 'cloze.txt')

# Construct command
cmd = f'decktor process "{input_filename}" "{output_filename}" --model "{model}" --batch-size {batch_size} --prompt "{prompt_file}"'

if limit > 0:
    cmd += f' --limit {limit}'

print(f"📝 Card Type: {card_type}")
print(f"Running: {cmd}\n")
!{cmd}

print(f"\n✅ Done! Saved to: {output_filename}")

In [ ]:
#@title 4. Download Result
from google.colab import files
files.download(output_filename)